# Ex05 - Filtros de aguçamento

## 1. Unsharp mask

Um filtro bastante utilizado para aguçar a imagem é denominado *unsharp mask*. Ele é capaz de realçar bordas calculando a diferença entre a imagem original e uma versão suavizada da imagem filtrada pela gaussiana. Para conseguir o realce de bordas, faça:

- Calcule primeiro a *unsharp mask* ($df$)
- Faça uma ponderação entre a imagem original e a imagem diferença: 
$$((1-k)*f + k*df)$$ onde $f$ é a imagem, $df$ é a *unsharp mask* e $k$ é o fator de ponderação 
- Mude o fator de ponderacao $k$ e veja o efeito na imagem final


In [ ]:
import sys, os
ia898path = os.path.abspath('../../')
if ia898path not in sys.path:
    sys.path.append(ia898path)
import ia898.src as ia

import numpy as np
import scipy.signal as sc
import scipy.ndimage as sn
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from math import sqrt, pi

In [ ]:
def border_highlight(img, df, k):
    return(((1 - k) * img) + (k * df))

In [ ]:
def gauss_kernel(kernel_size=7, sigma=1.0, mean=0):
    
    x = np.linspace(mean-1, mean+1, kernel_size)
    g_x = 1/(sqrt(2*pi)*sigma) * np.exp(-(x**2/2.*(sigma**2)))
    g_y = g_x.reshape(len(g_x),1)
    g_xy = g_x * g_y
#     plt.plot(x, g_x)
#     plt.show()
    return g_xy

Utilizando uma imagem maior, percebi que a ponderação e o filtro gaussiano devem ser maiores para que tenhamos diferenças visíveis. Neste caso, a imagem é relativamente pequena.

In [ ]:
f_ex1 = np.array(mpimg.imread('../data/retina.tif')).astype('float32')

gauss_kernel1_ex1 = gauss_kernel(3, 1, 0)

f_ex1_filtered = sc.convolve2d(f_ex1, gauss_kernel1_ex1, mode='same')

# f_ex1_filtered = ia.normalize(sn.gaussian_filter(f_ex1, 1))

n = 0
m = 200

df_ex1 = f_ex1 - f_ex1_filtered

fig1_ex1 = plt.figure(figsize=(14,14))

fig1_ex1.add_subplot(131).axis('off')
plt.title('Original')
plt.imshow(f_ex1[n:m,n:m], cmap='gray')

fig1_ex1.add_subplot(132).axis('off')
plt.title('Filtrado gaussian')
plt.imshow(f_ex1_filtered[n:m,n:m], cmap='gray')

fig1_ex1.add_subplot(133).axis('off')
plt.title('Original - filtrado')
plt.imshow(df_ex1[n:m,n:m], cmap='gray')

plt.show()

fig2_ex1 = plt.figure(figsize=(14,14))


fig2_ex1.add_subplot(131).axis('off')
plt.title('Ponderação 0.5')
plt.imshow((ia.normalize(border_highlight(f_ex1, df_ex1, .5))[n:m,n:m]), cmap='gray')

fig2_ex1.add_subplot(132).axis('off')
plt.title('Ponderação 1')
plt.imshow(border_highlight(f_ex1, df_ex1, 1)[n:m,n:m], cmap='gray')

fig2_ex1.add_subplot(133).axis('off')
plt.title('Ponderação 2')
plt.imshow(border_highlight(f_ex1, df_ex1, 2)[n:m,n:m], cmap='gray')

plt.show()

## 2. Filtro de Sobel

Existem vários filtros que procuram realçar as bordas da imagem. Um dos mais conhecidos é o Operador Sobel, composto por uma máscara vertical (Sv) e uma máscara horizontal (Sh).

In [ ]:
import numpy as np

Sv = np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
print('Sv =\n',Sv)

Sh = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
print('Sh =\n',Sh)



### 2.1 Implementar o operador Magnitude Sobel de uma imagem.

A função MagSobel a ser implementada possui como parâmetro a imagem de entrada e deve seguir a seguinte equação:

$$MagSobel = \sqrt{f_h^2 + f_v^2}$$

onde $f_h$ é a imagem de entrada convolvida com o operador de Sobel horizontal e $f_v$ é a imagem de entrada convolvida com o operador de Sobel vertical.

Existem alguns cuidados necessários:

- As operações devem todas serem feitas em ponto flutuante e os valores finais serão maiores de 255. Assim, a função que calcula a magnitude do gradiente Sobel é feita de acordo com a equação dada. 
- Lembre-se que para visualizar a imagem será necessário antes normalizar a imagem utilizando, por exemplo, o ianormalize. 
- Adicionalmente, como a máscara Sobel é 3x3, a imagem resultante terá altura e largura maiores que a original por 2x2 pixels já que a imagem resultante da convolução linear é a soma dos tamanhos em cada dimensão, menos 1.



In [ ]:
def MagSobel(img):

    Sv = np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
    Sh = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
    
    fh = sc.convolve2d(img, Sh, mode='same').astype('float')
    fv = sc.convolve2d(img, Sv, mode='same').astype('float')
    
    mag_sobel = np.sqrt((fv**2)+(fh**2))
    
    return mag_sobel, fh, fv

f = mpimg.imread('../data/retina.tif')
sobel = MagSobel(f)

fig = plt.figure(figsize=[5, 5])

plt.imshow(ia.normalize(sobel[0]), cmap='gray')
plt.show()

### 2.2 Implementar uma funcao que retorna o ângulo da borda de Sobel

Após implementar a função que retorna o ângulo da borda de Sobel:
- calcule o histograma da distribuição deste ângulo, somente para valores de magnitude de borda acima de um limiar $T$;
- visualize a imagem de ângulo utilizando uma tabela de cores circular para os ângulos e uma cor diferente para usar nos locais onde a magnitude for menor que $T$. Plote junto com a figura, a tabela de cores usada;


In [ ]:
def sobel_angle(fh, fv):
    return(np.arctan2(fv, fh))

**Fernando:** Imprimi os valores abaixo somente para ter certeza da faixa de ângulos resultantes na operação ângulos mod pi, para que só tivéssemos ângulos positivos, de 0 a 180 graus. Sabemos que utilizar abs não seria exatamente efetivo, pois em nosso caso, -pi/4 seria equivalente a 3/4 * pi, por exemplo.

In [ ]:
f = mpimg.imread('../data/retina.tif')
# f = mpimg.imread('../data/cameraman.tif')
sobel = MagSobel(f)
angles = (sobel_angle(sobel[1], sobel[2]))
print(angles.max())
print(angles.min())

print(sobel[0].max())

angles = angles % (pi)
print(angles.max())
print(angles.min())

**Fernando:** Pelo histograma da magnitude normalizada, podemos perceber que a maior quantidade de magnitudes está concentrada nos valores 50 e abaixo. Fui experimentando a partir de 50, e a imagem ficou mais interessante a partir de 130

In [ ]:
plt.plot(np.histogram(sobel[0], bins=1070)[0])
plt.show()

sobel_normalized = ia.normalize(sobel[0])
plt.plot(ia.histogram(sobel_normalized))
plt.show()

### Eu tinha feito primeiro a circunferência abaixo, a partir do exemplo no ia.sobel

In [ ]:
f2_ex2 = ia.circle([200,300], 90, [100,150])
sobel2_ex2 = MagSobel(f2_ex2)
f2_angles = sobel_angle(sobel2_ex2[1], sobel2_ex2[2])
f2_angles = f2_angles % pi

f2_mag_angles = np.select([sobel2_ex2[0] > 1], [f2_angles])

f2_angles_color = np.ones((f2_angles.shape[0], f2_angles.shape[1], 3))

f2_angles_color[:,:,0] = ia.normalize(f2_mag_angles, [0, 1])
f2_angles_color[:,:,1] = ia.normalize(f2_mag_angles, [0, 1])
f2_angles_color[:,:,2] = ia.normalize(f2_mag_angles, [0, 1])
f2_angles_color[:,:,1][f2_angles_color[:,:,0] != 0] = 1
f2_angles_color[:,:,2][f2_angles_color[:,:,0] != 0] = 1
print(f2_mag_angles.min())
plt.imshow(f2_mag_angles, cmap='gray')
plt.colorbar()
plt.show()

plt.imshow(colors.hsv_to_rgb(f2_angles_color), cmap='hsv', vmax=180)
plt.colorbar()
plt.show()

**Fernando:** Eu acho que não era bem essa tabela de cores que a professora queria, mas perdi muito tempo fazendo os outros exercícios. Então acredito que vou deixar desta forma.

In [ ]:
import matplotlib.colors as colors

mag_angles = np.select([sobel[0] > 130], [angles])

plt.figure(figsize=[7,7])
plt.imshow(ia.normalize(mag_angles, [0, 180]), cmap='gray', vmax=180)
plt.colorbar()
plt.show()

colored_angles = np.ones((mag_angles.shape[0], mag_angles.shape[1], 3))
colored_angles[:,:,0] = ia.normalize(mag_angles, [0, 1])
colored_angles[:,:,1] = ia.normalize(mag_angles, [0, 1])
colored_angles[:,:,2] = ia.normalize(mag_angles, [0, 1])

colored_angles[:,:,1][colored_angles[:,:,0] != 0] = 1
colored_angles[:,:,2][colored_angles[:,:,0] != 0] = 1

plt.figure(figsize=[7,7])

plt.imshow(colors.hsv_to_rgb(colored_angles), cmap='hsv', vmax=180)
plt.colorbar()
plt.show()



## 3. Propriedades da convolução

Realize experimentos para demostrar as propriedades da convolução:

- Comutativa $$f(x,y)*g(x,y) = g(x,y)*f(x,y)$$
- Associativa $$f(x,y)*[g(x,y)*h(x,y)] = [f(x,y)*g(x,y)]*h(x,y)$$

Crie um exemplo que demonstre como usar a propriedade associativa para realizar a filtragem de uma imagem por um filtro passa-faixa. (Dica: use uma máscara passa-baixas e uma máscara passa-altas).

**Fernando**: Fiz a convolução comutativa com ia.conv e scipy.signal.convolve. Isso porque eu sei que o numpy.convolve alterna qual array convolve em qual, dependendo do tamanho, porém não tenho certeza quanto ao scipy.  
Tirei a diferença entre os resultados nos dois casos, e foi zero.

In [ ]:
kernel_ex3 = gauss_kernel(7)

f_filtered_ex3_1 = ia.conv(f, kernel_ex3)

f_filtered_ex3_2 = ia.conv(kernel_ex3, f)

fig_ex3 = plt.figure(figsize=[10, 10])
fig_ex3.add_subplot(121).axis('off')
plt.imshow(f_filtered_ex3_1, cmap='gray')
fig_ex3.add_subplot(122).axis('off')
plt.imshow(f_filtered_ex3_2, cmap='gray')

plt.show()

print(np.sum(f_filtered_ex3_1 - f_filtered_ex3_2))


f_filtered_ex3_1 = sc.convolve(f, kernel_ex3)

f_filtered_ex3_2 = sc.convolve(kernel_ex3, f)

fig_ex3 = plt.figure(figsize=[10, 10])
fig_ex3.add_subplot(121).axis('off')
plt.imshow(f_filtered_ex3_1, cmap='gray')
fig_ex3.add_subplot(122).axis('off')
plt.imshow(f_filtered_ex3_2, cmap='gray')

plt.show()

print(np.sum(f_filtered_ex3_1 - f_filtered_ex3_2))

- Associativa $$f(x,y)*[g(x,y)*h(x,y)] = [f(x,y)*g(x,y)]*h(x,y)$$

In [ ]:
f_xy = f.copy()

# Laplaciano
g_xy = np.array([[0, 0, -1, 0, 0],
                 [0, -1, -2, -1, 0],
                 [-1,  -2,  16,  -2, -1],
                 [0, -1, -2, -1, 0],
                 [0, 0, -1, 0, 0]])

h_xy = np.ones((5, 5))/5*5


# f(x, y) * [g(x, y) * h(x, y)]
term1 = sc.convolve2d(g_xy, h_xy, mode='full')

result1 = sc.convolve2d(term1, f_xy, mode='full')

# g(x, y) * [f(x, y) * h(x, y)]
term2 = sc.convolve2d(f_xy, h_xy, mode='full')

result2 = sc.convolve2d(term2, g_xy, mode='full')

# h(x, y) * [f(x, y) * g(x, y)]
term3 = sc.convolve2d(f_xy, g_xy, mode='full')

result3 = sc.convolve2d(term3, h_xy, mode='full')

fig_ex3_3 = plt.figure(figsize=[12, 12])

fig_ex3_3.add_subplot(131)
plt.imshow(result1, cmap='gray')

fig_ex3_3.add_subplot(132)
plt.imshow(result2, cmap='gray')

fig_ex3_3.add_subplot(133)
plt.imshow(result2, cmap='gray')
plt.show()

print(np.sum(result1-result2))
print(np.sum(result1-result3))